<div align="center">

# RIO Airbnb - Data Cleaning 
**Latest Update:** _10th December 2022_
</div>

---
#### _Import required libraries_

In [1]:
import sys, os
sys.path.append(os.path.abspath("../"))

import numpy as np
import pandas as pd
import scipy as scp
from sklearn.preprocessing import OneHotEncoder

import airbnblib.cleaning as cln

#### *Import initial dataset*

In [2]:
listings_df = pd.read_parquet("../dataset_zipped/listings.parquet")

#### *Remove unnecessary data columns*

In [3]:
listings_df = listings_df.drop([
    'listing_url', 'scrape_id', 'last_scraped', 'source', 'host_id', 'host_url', 'host_name', 'host_location', 'picture_url', 'host_thumbnail_url', 'host_picture_url', 'host_verifications', 
    'host_identity_verified', 'neighbourhood', 'property_type', 'bathrooms', 'calendar_updated', 'calendar_last_scraped', 'minimum_minimum_nights', 'maximum_minimum_nights',
    'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'first_review', 'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms',
    'calculated_host_listings_count_shared_rooms', 'host_listings_count', 'host_total_listings_count'
], axis=1)   

#### *Fill in & replace selected value columns*

In [4]:
NULL_VALUES = {
    'license': 'none',
    'host_response_time': 'does not respond',
    'reviews_per_month': 0,
    'bathrooms_text': "0 private baths"
}

# Fill in null values in certain columns
for nul in NULL_VALUES:
    listings_df.loc[listings_df[nul].isnull(), nul] = NULL_VALUES[nul]

# Convert all values in 'license' to categorical values
listings_df.loc[listings_df["license"] == "Exempt", "license"] = "exempted"
listings_df.loc[~listings_df['license'].str.contains('none|exempted'), 'license'] = "have"

# Clean the 'price' column by removing the '$' and the ',' symbols as well as converting all values to "float" data type
listings_df['price'] = listings_df['price'].map(lambda x: float(x.lstrip('$').replace(",", "")))

#### *Convert values from `bathrooms_text` to individual columns of `bathroom_num (float)` and `bathroom_type (vectors)`*

In [5]:
# Divide `bathrooms_text` column into `num` & `type`
NEW_COLUMNS = ['bathroom_type', 'bathroom_num']
for col in NEW_COLUMNS:
    listings_df[col] = listings_df["bathrooms_text"].map(lambda x: cln.bath_clean(x)[col])

listings_df = listings_df.join(pd.get_dummies(listings_df["bathroom_type"])).drop(["bathrooms_text"], axis=1)

#### *Convert `true/false` categorical columns to one-hot vectors*

In [6]:
TRUE_FALSE = ['has_availability', 'instant_bookable', "host_is_superhost"]
ONE_HOT_VECTORS = ['license']

# Convert all t/f columns to true/false & add them to the ONE_HOT_VECTORS list
for tf in TRUE_FALSE:
    listings_df.loc[listings_df[tf] == 't', tf] = f"{tf}_true"
    listings_df.loc[listings_df[tf] == 'f', tf] = f"{tf}_false"
    ONE_HOT_VECTORS.append(tf)


# Convert all categorical columns into one-hot vectors
listings_df = listings_df.join(pd.get_dummies(listings_df[ONE_HOT_VECTORS])).drop(ONE_HOT_VECTORS, axis=1)

#### *Remove all $0 listings and listings with z-score over +/-2.5*

In [7]:
# Add a 'price_zscore' column of all listings with price other than $0
listings_df['price_zscore'] = scp.stats.zscore(listings_df[listings_df['price'] != 0]['price'])

# Remove all rows with a price z-score of more than +-2.5
listings_df = listings_df[abs(listings_df["price_zscore"]) < 2.5]

listings_df.describe()

,id,latitude,longitude,accommodates,bedrooms,beds,price,minimum_nights,maximum_nights,maximum_nights_avg_ntm,...,license_exempted,license_have,license_none,has_availability_has_availability_false,has_availability_has_availability_true,instant_bookable_instant_bookable_false,instant_bookable_instant_bookable_true,host_is_superhost_host_is_superhost_false,host_is_superhost_host_is_superhost_true,price_zscore
count,4.521400e+04,45214.000000,45214.000000,45214.000000,41484.000000,44445.000000,45214.000000,45214.000000,45214.000000,4.521400e+04,...,45214.000000,45214.000000,45214.000000,45214.000000,45214.000000,45214.000000,45214.000000,45214.000000,45214.000000,45214.000000
mean,1.637819e+17,33.990167,-118.237441,4.067789,1.825523,2.280031,232.262858,17.878113,579.489295,4.467158e+05,...,0.023400,0.237471,0.739129,0.078560,0.921440,0.619498,0.380502,0.702327,0.296921,-0.080546
std,2.796691e+17,0.190991,0.235640,2.831597,1.164331,1.716960,259.203331,30.248155,664.987864,3.023124e+07,...,0.151171,0.425538,0.439114,0.269053,0.269053,0.485516,0.485516,0.457240,0.456907,0.359869
min,1.090000e+02,33.338480,-118.961700,1.000000,1.000000,1.000000,10.000000,1.000000,1.000000,1.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.389128
25%,2.381052e+07,33.892572,-118.394588,2.000000,1.000000,1.000000,90.000000,2.000000,90.000000,3.650000e+02,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,-0.278058
50%,4.517365e+07,34.043690,-118.312715,3.000000,1.000000,2.000000,150.000000,7.000000,365.000000,1.125000e+03,...,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,-0.194757
75%,5.487822e+17,34.099947,-118.080682,6.000000,2.000000,3.000000,264.000000,30.000000,1125.000000,1.125000e+03,...,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.036483
max,7.118480e+17,34.822060,-117.627790,16.000000,24.000000,32.000000,2085.000000,1124.000000,99999.000000,2.147484e+09,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.491728


#### *Convert all `amenities` rows to `list[str]` & eliminate all values within each amenities list that are less than 50% frequency*

In [22]:
# For more documentation on these functions, check `airbnblib/cleaning.py`
listings_df['amenities'] = [cln.amenities_clean(row) for row in listings_df['amenities']]
all_amenities = cln.all_amenities(listings_df['amenities'])

for dummy in all_amenities:
    new_col = [
        1 if dummy in each_listing else 0
        for each_listing in listings_df['amenities']
    ]
    if sum(new_col) > 22901:
        listings_df["amenities_" + dummy.lower().replace(' ','_')] = new_col
    else:
        all_amenities.remove(dummy)
listings_df.pop('amenities')

#### *Extract listings zip code using using `latitude` and `longitude` columns*
*Caution:* This process will take a very long time, possibly over half a day. It is best advised to only run this process once

In [12]:
# For more documentation on these functions, check `airbnblib/cleaning.py`
# listings_df["address"] = [
#     cln.get_zip_code(listings_df['latitude'].to_list()[i], listings_df['longitude'].to_list()[i])
#     for i in range(len(listings_df['latitude'].to_list()))
# ]

listings_df["address"] = "{0:}, {1}".format(listings_df['latitude'], listings_df['longitude'])

In [13]:
listings_df['address'].head()

0    0        33.99211\n1        34.12206\n2       ...
1    0        33.99211\n1        34.12206\n2       ...
2    0        33.99211\n1        34.12206\n2       ...
3    0        33.99211\n1        34.12206\n2       ...
4    0        33.99211\n1        34.12206\n2       ...
Name: address, dtype: object

<h2 align="center">Tools for Referencing</h2>

---
#### **Export DataFrame to CSV**
Please close Excel or any other application that's opening the destination CSV before exporting, otherwise you'll get `[Errno 13] Permission Denied`

In [24]:
from datetime import datetime

today = str(datetime.now()).split(" ")[0].replace('-', '')
listings_df.to_csv(f"dataset/dataset_cleaned_{today}.csv", index=False)

#### **Export DataFrame to Parquet**

In [13]:
listings_df.to_parquet("../dataset_zipped/listings.parquet")

#### **Check for unique variables in each column**

In [ ]:
col_name = input("Enter column name here: ")
display_list = []
for row in listings_df[col_name]: 
    if row not in display_list:
        display_list.append(row)

display_list

#### **Find percentage in of certain values**

In [ ]:
PERCENTAGES = {
    "sum": 0
}

listings_df.loc[listings_df["host_identity_verified"].isnull(), "host_identity_verified"] = "na"

for row in listings_df['host_identity_verified']:
    if row not in PERCENTAGES:
        PERCENTAGES[row] = 0
    PERCENTAGES[row] += 1
    PERCENTAGES["sum"] += 1
    
print("Percentage: " + str(
    PERCENTAGES["t"] / PERCENTAGES['sum']
))

#### **Find data type of certain columns**

In [ ]:
col_name = input("Enter column name here: ")
print(type(listings_df[col_name].to_list()[0]))

In [2]:
listings_df.info()

NameError: name 'listings_df' is not defined